In [13]:
import torch
import numpy as np
import torch.nn.functional as F
import torch.nn as A
import pandas as pd
from sklearn.manifold import TSNE
import sklearn.manifold 
from matplotlib import pyplot as plt
import re
from numpy  import array
from scipy import stats
# from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity

# PREPROCESSING

In [2]:
corpus_filename = '../data/extracted_emoji_sequences.txt' 

In [3]:
corpus = open(corpus_filename).read().splitlines()

In [4]:
def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens

In [5]:
def onlyEmojiSequences(tokens):
    threshold_emojis = [x for x in tokens if len(x) > 1]
    return threshold_emojis

In [6]:
tokenized_corpus = tokenize_corpus(corpus)
emojiSequences = onlyEmojiSequences(tokenized_corpus)

In [7]:
vocabulary = []
for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)

# this is just the very basic translation both ways plus the length of vocabulary

In [8]:
window_size = 2
idx_pairs = []
# for each sentence
for sentence in tokenized_corpus:
    indices = [word2idx[word] for word in sentence]
    # for each word, threated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array

# sole purpose of this is to have pairs! of target and context word
# super simple once you have figured out the code!

In [9]:
def get_input_layer(word_idx):
    x = torch.zeros(vocabulary_size).float()
    x[word_idx] = 1.0
    return x

# this is a one hot encoded something

# TRAINING

In [10]:
dtype = torch.float 
dimensionSize = 300
num_epochs = 1
learning_rate = 0.001


firstLayer = torch.randn(dimensionSize, vocabulary_size, requires_grad=True).float()
secondLayer = torch.randn(vocabulary_size, dimensionSize, requires_grad=True).float()

for epo in range(num_epochs):
    loss_val = 0
    for data, target in idx_pairs:


        # one hot encoded tensor
        x = get_input_layer(data).float()
        # target word 
        y_true = torch.from_numpy(np.array([target])).long()

        # Hidden Layer: gradient magic happening ...
        z1 = torch.matmul(firstLayer, x)
        z2 = torch.matmul(secondLayer, z1)

        # introducing non-linearity
        softmax = A.LogSoftmax(dim=0)
        soft_max_output = softmax(z2)

        # neg_log_likelihood
        loss = F.nll_loss(soft_max_output.view(1,-1), y_true)
        loss_val += loss  # this might be to please the  framework, and adding stuff to the gradient calculator
        
        # propagating it back
        loss.backward()
        
        # updating the weights of both layers
        firstLayer.data -= learning_rate * firstLayer.grad.data
        secondLayer.data -= learning_rate * secondLayer.grad.data

        # set the gradients to zero for next iteration
        firstLayer.grad.data.zero_()
        secondLayer.grad.data.zero_()
        
    # this keeps track of the loss, hopefully it does converge
#     if epo % 1000 == 0:    
#         print(f'Loss at epo {epo}: {loss_val/len(idx_pairs)}')

    print(f'Loss at epo {epo}: {loss_val/len(idx_pairs)}')





KeyboardInterrupt: 

# EVALUATING TRAINED MODEL

In [ ]:
word2idx

In [ ]:
corpus_filename = './data/EmoSim508.json'
corpus = open(corpus_filename).read()

In [ ]:
myEmojiPairScores = []
approved_annotator_results = []
all_annotator_results = list(array(re.findall('(?<=_Annotator_Agreement": )(.*?)(?=\})', corpus)))

# get all unicodes from all 508 pairs
unicodeEmojiPairs = re.findall('(?<=unicodeshort": ")(.*?)(?=\")', corpus)

for x in range(508):
    emoji1 = unicodeEmojiPairs.pop()
    emoji2 = unicodeEmojiPairs.pop()
    respective_annotator_result = all_annotator_results.pop(0)
    if (emoji1 and emoji2 in word2idx) and ((len(emoji1) < 9 ) and (len(emoji2) < 9)):
        try:
            myEmojiPairScores.append(distance.euclidean(secondLayer.detach().numpy()[word2idx[emoji1]], secondLayer.detach().numpy()[word2idx[emoji2]]))
            approved_annotator_results.append(respective_annotator_result)
        except:
            print("didn't work")
    

In [ ]:
print(stats.spearmanr(myEmojiPairScores, approved_annotator_results))

In [ ]:
# extracts the 508 Annotator Results as the Gold-Standard
corpus_filename = '../data/EmoSim508.json'
corpus = open(corpus_filename).read()
annotator_similarity_score_508 = list(array(re.findall('(?<=_Annotator_Agreement": )(.*?)(?=\})', corpus)))

# glyph_pairs_1016
unicode_pairs_1016 = re.findall('(?<=unicodelong": "\\\)(.*?)(?=")', corpus)    
glyph_pairs_1016 = [codecs.decode(unicode_pairs_1016[x].replace(str('\\\\'),str('\\')).replace('_',''), 'unicode_escape') for x in range(len(unicode_pairs_1016))]

# computation of SpearRank
goldstandard = []
selftrained = []
for x in range(len(annotator_similarity_score_508)):
    cosineSimilarity = None
    
    emoji1 = glyph_pairs_1016.pop(0)
    emoji2 = glyph_pairs_1016.pop(0)
    
    try:
        cosineSimilarity = cosine_similarity([1, 0, -1], [-1,-1, 0])
    except:
        print('the cosine similarity between ' + emoji1 + ' and ' + emoji2 + ' could not be computed.')
    
    if(cosineSimilarity is not None):
        selftrained.append(cosineSimilarity)
        goldstandard.append(annotator_similarity_score_508.pop(0))

spearmanRank = stats.spearmanr(goldstandard, selftrained)

print('Der Spearman Rank Correlation Coefficient is {}'.format(spearmanRank))